In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

# heat fluxes ds

## gridding funcs

In [3]:
# 3d gridding funcs
def grid_lat_3d(dsgpd_ln,gs=1):
    lat_min = -80
    lat_max = -30
    lat = np.arange(lat_min,lat_max+gs,gs)
    lat_labels = range(0,lat_max-lat_min,gs)
    
    return dsgpd_ln.groupby_bins('lat',lat,
                       labels=lat_labels,
                       restore_coord_dims=True).median(skipna=True,dim='time')
    
def grid_lon_3d(dsgpd_t,gs=1):
    # define lon min and max resp
    lon_min = -180
    lon_max = 180
    lon = np.arange(lon_min,lon_max+gs,gs)
    lon_labels = range(0,lon_max-lon_min,gs)

    return dsgpd_t.groupby_bins('lon',lon,
                       labels=lon_labels,
                       restore_coord_dims=True).apply(grid_lat_3d)

def grid_month_3d(dsgpd_yr,gs=1):
    """grid dataset onto a monthly climatology"""
    return dsgpd_yr.groupby('time.month').apply(grid_lon_3d)

def grid_var_3d(dsvar,gs=1):
    """grid 1x1° onto year-month index time series"""
    return dsvar.groupby('time.year').apply(grid_month_3d)

# 4d gridding funcs

def grid_lat_4d(dsgpd_ln,gs=1):
    lat_min = -80
    lat_max = -30
    lat = np.arange(lat_min,lat_max+gs,gs)
    lat_labels = range(0,lat_max-lat_min,gs)
    
    return dsgpd_ln.groupby_bins('lat',lat,
                       labels=lat_labels,
                       restore_coord_dims=True).median(skipna=True,dim='n_prof')
    
def grid_lon_4d(dsgpd_t,gs=1):
    # define lon min and max resp
    lon_min = -180
    lon_max = 180
    lon = np.arange(lon_min,lon_max+gs,gs)
    lon_labels = range(0,lon_max-lon_min,gs)

    return dsgpd_t.groupby_bins('lon',lon,
                       labels=lon_labels,
                       restore_coord_dims=True).apply(grid_lat_4d)

def grid_month_4d(dsgpd_yr,gs=1):
    return dsgpd_yr.groupby('time.month').apply(grid_lon_4d)

def grid_var_4d(dsvar,gs=1):
    return dsvar.groupby('time.year').apply(grid_month_4d)

## grid datasets

In [68]:
hf = xr.open_mfdataset('/home/theospira/notebooks/data/ERA5_heat_flux/6_hourly/**.nc').rename({'longitude':'lon','latitude':'lat'})
hf = hf.sel(lat=slice(-30,-79))
hf

<xarray.Dataset>
Dimensions:  (lon: 1442, lat: 197, time: 26300)
Coordinates:
  * lon      (lon) float32 -180.0 -179.8 -179.5 -179.2 ... 179.5 179.8 180.0
  * lat      (lat) float32 -30.0 -30.25 -30.5 -30.75 ... -78.5 -78.75 -79.0
  * time     (time) datetime64[ns] 2004-01-01 ... 2021-12-31T18:00:00
Data variables:
    slhf     (time, lat, lon) float32 dask.array<chunksize=(1464, 197, 721), meta=np.ndarray>
    sshf     (time, lat, lon) float32 dask.array<chunksize=(1464, 197, 721), meta=np.ndarray>
    ssr      (time, lat, lon) float32 dask.array<chunksize=(1464, 197, 721), meta=np.ndarray>
    str      (time, lat, lon) float32 dask.array<chunksize=(1464, 197, 721), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-08 09:41:00 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [79]:
def coarsen_ds(ds,gs=1,lat2b=50,lon2b=360):
    """
    coarsen a gridded dataset to a 1x1° lat-lon gridspace.
    ------
    params:
    ds: xarray.Dataset, 
        desired dataset to coarsen
    gs: grid size
        default 1
    lat2b: size of latitude grid to be [-79,-30]
           default 30
    lon2b: size of longitude grid to be [-180,180]
           default 360
    
    """
    ds = (ds.coarsen(lat=int(ds.lat.size*gs/lat2b),boundary='trim').mean(
            ).coarsen(lon=int(ds.lon.size*gs/lon2b),boundary='trim').mean())
    return ds

In [70]:
hf.lat.size/33

5.96969696969697

In [81]:
#hf = 
coarsen_ds(hf)

<xarray.Dataset>
Dimensions:  (time: 26300, lat: 65, lon: 360)
Coordinates:
  * lon      (lon) float32 -179.6 -178.6 -177.6 -176.6 ... 177.1 178.1 179.1
  * lat      (lat) float32 -30.25 -31.0 -31.75 -32.5 ... -76.75 -77.5 -78.25
  * time     (time) datetime64[ns] 2004-01-01 ... 2021-12-31T18:00:00
Data variables:
    slhf     (time, lat, lon) float32 dask.array<chunksize=(1464, 65, 180), meta=np.ndarray>
    sshf     (time, lat, lon) float32 dask.array<chunksize=(1464, 65, 180), meta=np.ndarray>
    ssr      (time, lat, lon) float32 dask.array<chunksize=(1464, 65, 180), meta=np.ndarray>
    str      (time, lat, lon) float32 dask.array<chunksize=(1464, 65, 180), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-08 09:41:00 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [100]:
idx = np.unique(hf.isel(lon=720).slhf == hf.isel(lon=721).slhf, return_index=True)

In [107]:
idx

(array([False,  True]), array([   0, 1623]))

In [106]:
hf.isel(lon=721).slhf

<xarray.DataArray 'slhf' (time: 26300, lat: 197)>
dask.array<getitem, shape=(26300, 197), dtype=float32, chunksize=(1464, 197), chunktype=numpy.ndarray>
Coordinates:
    lon      float32 0.0
  * lat      (lat) float32 -30.0 -30.25 -30.5 -30.75 ... -78.5 -78.75 -79.0
  * time     (time) datetime64[ns] 2004-01-01 ... 2021-12-31T18:00:00
Attributes:
    units:          J m**-2
    long_name:      Surface latent heat flux
    standard_name:  surface_upward_latent_heat_flux

In [104]:
hf.isel(lon=720).slhf[0].load()

<xarray.DataArray 'slhf' (lat: 197)>
array([-3.62235625e+05, -3.56245000e+05, -3.50034500e+05, -3.35195312e+05,
       -3.13925875e+05, -2.92711312e+05, -2.71551750e+05, -2.51711250e+05,
       -2.30991375e+05, -2.07083812e+05, -1.84165500e+05, -1.66633312e+05,
       -1.53333000e+05, -1.53882625e+05, -1.69601125e+05, -1.99939000e+05,
       -2.30057062e+05, -2.48303750e+05, -2.46490062e+05, -2.17801000e+05,
       -1.78119938e+05, -1.62676188e+05, -1.70095812e+05, -1.75152125e+05,
       -1.79988562e+05, -1.86473812e+05, -1.84715125e+05, -1.78559625e+05,
       -1.92464500e+05, -2.09447062e+05, -2.12469875e+05, -2.09996688e+05,
       -1.74657438e+05, -1.26512562e+05, -8.93596875e+04, -6.92993125e+04,
       -5.97363750e+04, -5.40205000e+04, -4.71505000e+04, -3.80821250e+04,
       -3.09922500e+04, -2.79695000e+04, -2.97831250e+04, -3.21465000e+04,
       -3.22013750e+04, -3.02228750e+04, -2.64306250e+04, -1.67576250e+04,
        1.54400000e+03,  1.88563750e+04,  3.00132500e+04,  3.58938750e+04,
        4.25990000e+04,  5.45802500e+04,  7.11232500e+04,  8.72265000e+04,
        9.45911250e+04,  8.40937500e+04,  5.21071250e+04, -2.79787500e+03,
       -5.17121250e+04, -8.03462500e+04, -9.22176250e+04, -9.93074375e+04,
       -1.07826250e+05, -1.10134562e+05, -1.16674750e+05, -1.24698938e+05,
       -1.38933500e+05, -1.60203000e+05, -1.81582438e+05, -1.98949750e+05,
       -2.12305000e+05, -2.22912250e+05, -2.32365375e+05, -2.41488688e+05,
       -2.44566438e+05, -2.40004812e+05, -2.26429688e+05, -2.03786188e+05,
...
       -7.39159375e+04, -6.47926250e+04, -5.12725000e+04, -3.89065000e+04,
       -2.63207500e+04, -1.58783750e+04, -9.55787500e+03, -6.31525000e+03,
       -5.05125000e+03, -4.28175000e+03, -2.79787500e+03, -1.14900000e+03,
       -8.19250000e+02, -1.86350000e+03, -4.00700000e+03, -6.31525000e+03,
       -7.30462500e+03, -6.31525000e+03, -4.66650000e+03, -3.07262500e+03,
       -2.96275000e+03, -3.51237500e+03, -4.39175000e+03, -5.05125000e+03,
       -5.32600000e+03, -5.27100000e+03, -4.88637500e+03, -4.66650000e+03,
       -4.17187500e+03, -3.95200000e+03, -4.17187500e+03, -4.61150000e+03,
       -4.22687500e+03, -3.56725000e+03, -3.40237500e+03, -2.85287500e+03,
       -4.98750000e+01,  5.22625000e+03,  5.11637500e+03,  7.53462500e+03,
        6.76512500e+03,  5.55600000e+03,  6.71025000e+03,  7.36975000e+03,
        4.29200000e+03,  2.42337500e+03, -4.11687500e+03, -7.46950000e+03,
       -6.15037500e+03, -3.84212500e+03, -1.75362500e+03, -1.14900000e+03,
       -1.47887500e+03,  1.10425000e+03,  3.46762500e+03,  5.06137500e+03,
        5.61100000e+03,  5.44612500e+03,  4.62175000e+03,  3.19275000e+03,
        2.09362500e+03,  2.31337500e+03,  3.19275000e+03,  3.63250000e+03,
        4.23700000e+03,  4.78662500e+03,  5.44612500e+03,  6.38050000e+03,
        7.69950000e+03,  9.23837500e+03,  1.07772500e+04,  1.24810000e+04,
        1.46793750e+04,  1.62182500e+04,  1.75922500e+04,  1.83067500e+04,
        1.81968750e+04], dtype=float32)
Coordinates:
    lon      float32 0.0
  * lat      (lat) float32 -30.0 -30.25 -30.5 -30.75 ... -78.5 -78.75 -79.0
    time     datetime64[ns] 2004-01-01
Attributes:
    units:          J m**-2
    long_name:      Surface latent heat flux
    standard_name:  surface_upward_latent_heat_flux

In [105]:
hf.isel(lon=721).slhf[0].load()

<xarray.DataArray 'slhf' (lat: 197)>
array([-3.62255125e+05, -3.56197312e+05, -3.50007812e+05, -3.35192500e+05,
       -3.13924250e+05, -2.92721875e+05, -2.71519500e+05, -2.51699938e+05,
       -2.30958500e+05, -2.07122250e+05, -1.84207875e+05, -1.66627000e+05,
       -1.53326125e+05, -1.53853000e+05, -1.69590125e+05, -1.99945000e+05,
       -2.30102500e+05, -2.48341812e+05, -2.46432250e+05, -2.17789312e+05,
       -1.78084250e+05, -1.62676250e+05, -1.70116875e+05, -1.75187000e+05,
       -1.79993750e+05, -1.86512500e+05, -1.84734625e+05, -1.78545125e+05,
       -1.92438625e+05, -2.09492688e+05, -2.12455750e+05, -2.09953625e+05,
       -1.74660250e+05, -1.26526875e+05, -8.93240000e+04, -6.93068750e+04,
       -5.96933750e+04, -5.40306250e+04, -4.71826250e+04, -3.80958750e+04,
       -3.09845000e+04, -2.79556250e+04, -2.97335000e+04, -3.21038750e+04,
       -3.21697500e+04, -3.01943750e+04, -2.64411250e+04, -1.67618750e+04,
        1.54337500e+03,  1.88607500e+04,  2.99887500e+04,  3.59148750e+04,
        4.26311250e+04,  5.46151250e+04,  7.10766250e+04,  8.72088750e+04,
        9.45835000e+04,  8.41140000e+04,  5.21130000e+04, -2.80250000e+03,
       -5.17260000e+04, -8.03031250e+04, -9.22212500e+04, -9.93325000e+04,
       -1.07826625e+05, -1.10131250e+05, -1.16650000e+05, -1.24683250e+05,
       -1.38971750e+05, -1.60174125e+05, -1.81574000e+05, -1.98957375e+05,
       -2.12258250e+05, -2.22925250e+05, -2.32407062e+05, -2.41493812e+05,
       -2.44522750e+05, -2.39979375e+05, -2.26415125e+05, -2.03764125e+05,
...
       -7.39160000e+04, -6.47635000e+04, -5.12651250e+04, -3.88860000e+04,
       -2.63095000e+04, -1.58400000e+04, -9.51875000e+03, -6.35812500e+03,
       -5.04125000e+03, -4.31700000e+03, -2.80250000e+03, -1.15637500e+03,
       -8.27125000e+02, -1.88062500e+03, -4.05362500e+03, -6.35812500e+03,
       -7.28000000e+03, -6.35812500e+03, -4.64612500e+03, -3.06587500e+03,
       -2.93425000e+03, -3.52675000e+03, -4.44862500e+03, -5.04125000e+03,
       -5.30462500e+03, -5.23875000e+03, -4.84375000e+03, -4.64612500e+03,
       -4.18525000e+03, -3.98775000e+03, -4.18525000e+03, -4.58037500e+03,
       -4.25112500e+03, -3.59262500e+03, -3.39512500e+03, -2.86837500e+03,
       -3.70000000e+01,  5.23075000e+03,  5.09900000e+03,  7.53525000e+03,
        6.81100000e+03,  5.55987500e+03,  6.74512500e+03,  7.33775000e+03,
        4.30887500e+03,  2.39937500e+03, -4.11937500e+03, -7.47750000e+03,
       -6.16062500e+03, -3.85600000e+03, -1.74900000e+03, -1.15637500e+03,
       -1.48562500e+03,  1.08237500e+03,  3.45287500e+03,  5.03312500e+03,
        5.62575000e+03,  5.42825000e+03,  4.63812500e+03,  3.18950000e+03,
        2.13600000e+03,  2.33350000e+03,  3.18950000e+03,  3.65037500e+03,
        4.24300000e+03,  4.76975000e+03,  5.42825000e+03,  6.41587500e+03,
        7.66700000e+03,  9.24725000e+03,  1.07617500e+04,  1.24737500e+04,
        1.46466250e+04,  1.61611250e+04,  1.76097500e+04,  1.83340000e+04,
        1.82023750e+04], dtype=float32)
Coordinates:
    lon      float32 0.0
  * lat      (lat) float32 -30.0 -30.25 -30.5 -30.75 ... -78.5 -78.75 -79.0
    time     datetime64[ns] 2004-01-01
Attributes:
    units:          J m**-2
    long_name:      Surface latent heat flux
    standard_name:  surface_upward_latent_heat_flux

In [96]:
hf.isel(lon=slice(720,722))

<xarray.Dataset>
Dimensions:  (lon: 2, lat: 197, time: 26300)
Coordinates:
  * lon      (lon) float32 0.0 0.0
  * lat      (lat) float32 -30.0 -30.25 -30.5 -30.75 ... -78.5 -78.75 -79.0
  * time     (time) datetime64[ns] 2004-01-01 ... 2021-12-31T18:00:00
Data variables:
    slhf     (time, lat, lon) float32 dask.array<chunksize=(1464, 197, 1), meta=np.ndarray>
    sshf     (time, lat, lon) float32 dask.array<chunksize=(1464, 197, 1), meta=np.ndarray>
    ssr      (time, lat, lon) float32 dask.array<chunksize=(1464, 197, 1), meta=np.ndarray>
    str      (time, lat, lon) float32 dask.array<chunksize=(1464, 197, 1), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-08 09:41:00 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [88]:
np.where(hf.lon.diff('lon')==0)

(array([720]),)

In [83]:
np.unique(hf.lon.diff('lon'))

array([0.  , 0.25], dtype=float32)

In [80]:
np.unique(coarsen_ds(hf).lon.diff('lon'))

array([0.8125, 0.9375, 1.    ], dtype=float32)

In [6]:
x    = 1 
time = np.asarray(pd.date_range("2004-01-01","2021-12-31",freq="1M"))
p    = xr.open_dataset('/home/theospira/notebooks/data/SO_2004-2021_2dbar_interp_w_mld.nc').pres.data
lat  = np.arange(-79,-30+x,x)
lon  = np.arange(-180,180,x)

ds_grid = xr.Dataset(
            data_vars = dict(
                            slhf = (['time','lon','lat'], dvars_4d[0]),
                            sshf = (['time','lon','lat'], dvars_4d[1]),
                            ssr  = (['time','lon','lat'], dvars_4d[2]),
                            str  = (['time','lon','lat'], dvars_4d[3]),
                            ),
            coords   = dict(
                            time    = (['time'], time),
                            lon     = (['lon'], lon),
                            lat     = (['lat'], lat),
                            ),
            attrs    = dict(description=str(
    '1 degree grid of median along 2dbar pressure interpolated monthly time series data from Argo, MEOP, SOCCOM, CTDs, and Gliders.'))
            )

# add mask
ds_grid['mask'] = xr.open_dataset('/home/theospira/notebooks/2m_interp/interp/3_part_mask.nc').mask
ds_grid["mask"].attrs["legend"] = "0 = Subantarctic Zone (SAZ). \n 1 = ACC Zone (ACCZ). \n 2 = Sub-ACC Zone (SACCZ)."

In [11]:
hfy = hf.groupby('time.year')

hfm = []
for i in tqdm(list(hfy.groups.keys())):
    hfm += hfy[i].groupby('time.month')[i],

In [ ]:
# load ssh
ssh = xr.open_dataset('/home/theospira/notebooks/data/Copernicus/ssh_monthly_climatology_2004-2021.nc')
# load sea ice
si = xr.open_dataset('/home/theospira/notebooks/data/meereisportal_sea_ice/sic_monthly_time_series.nc.nc')
